In [239]:
from itertools import pairwise
lmap = lambda f,i : list(map(f, i))

In [240]:
with open("test_input.txt", "r") as f:
    test_input = f.read().split("\n\n")

with open("input.txt", "r") as f:
    real_input = f.read().split("\n\n")

test_input

['seeds: 79 14 55 13',
 'seed-to-soil map:\n50 98 2\n52 50 48',
 'soil-to-fertilizer map:\n0 15 37\n37 52 2\n39 0 15',
 'fertilizer-to-water map:\n49 53 8\n0 11 42\n42 0 7\n57 7 4',
 'water-to-light map:\n88 18 7\n18 25 70',
 'light-to-temperature map:\n45 77 23\n81 45 19\n68 64 13',
 'temperature-to-humidity map:\n0 69 1\n1 0 69',
 'humidity-to-location map:\n60 56 37\n56 93 4']

In [241]:
def get_seeds(sections):
    return lmap(int, sections[0].split()[1:])

get_seeds(test_input)

[79, 14, 55, 13]

In [242]:
def split_section(section):
    range_lines = section.split("\n")[1:]
    return [lmap(int, line.split()) for line in range_lines]

split_section(test_input[2])

[[0, 15, 37], [37, 52, 2], [39, 0, 15]]

In [243]:
def convert_to_ranges(split_section):
    return sorted([((s, s+l-1), (d, d+l-1)) for d,s,l in split_section], key=lambda r: r[0])

convert_to_ranges(split_section(test_input[2]))

[((0, 14), (39, 53)), ((15, 51), (0, 36)), ((52, 53), (37, 38))]

In [244]:
def find_range(id, ranges):
    possible_ranges = [((a1,a2), (b1, b2)) for ((a1,a2),(b1,b2)) in ranges if a1<=id<=a2]
    return possible_ranges[0] if possible_ranges else ((id, id), (id, id))

def convert_value(id, ranges):
    (s1, _), (d1, d2) = find_range(id, ranges)
    return id - s1 + d1

convert_value(79, convert_to_ranges(split_section(test_input[1])))

81

In [245]:
def get_sequential_ranges(sections):
    return [convert_to_ranges(split_section(section)) for section in sections[1:]]

get_sequential_ranges(test_input)

[[((50, 97), (52, 99)), ((98, 99), (50, 51))],
 [((0, 14), (39, 53)), ((15, 51), (0, 36)), ((52, 53), (37, 38))],
 [((0, 6), (42, 48)),
  ((7, 10), (57, 60)),
  ((11, 52), (0, 41)),
  ((53, 60), (49, 56))],
 [((18, 24), (88, 94)), ((25, 94), (18, 87))],
 [((45, 63), (81, 99)), ((64, 76), (68, 80)), ((77, 99), (45, 67))],
 [((0, 68), (1, 69)), ((69, 69), (0, 0))],
 [((56, 92), (60, 96)), ((93, 96), (56, 59))]]

In [246]:
def follow_ranges_list(id, ranges_list):
    for ranges in ranges_list:
        id = convert_value(id, ranges)
    return id

follow_ranges_list(13, get_sequential_ranges(test_input))

35

In [247]:
def get_score_1(sections):
    seeds = get_seeds(sections)
    sequential_ranges = get_sequential_ranges(sections)
    locations = [follow_ranges_list(seed, sequential_ranges) for seed in seeds]
    return min(locations)

get_score_1(test_input)

35

In [248]:
get_score_1(real_input)

323142486

In [249]:
def get_seed_ranges(sections):
    part_1_seeds = get_seeds(sections)
    range_starts = part_1_seeds[0::2]
    range_lengths = part_1_seeds[1::2]
    return [((s, s+l-1), (s, s+l-1)) for s,l in zip(range_starts, range_lengths)]

get_seed_ranges(test_input)

[((79, 92), (79, 92)), ((55, 67), (55, 67))]

In [250]:
def overlaps(range1, range2):
    do_swap = range1[0] > range2[0]
    lower_range = range2 if do_swap else range1
    upper_range = range1 if do_swap else range2
    return lower_range[1] >= upper_range[0]

overlaps((4,7), (1,4))

True

In [251]:
def clip_range_map_input(range, range_map):
    rl, ru = range
    (s1, e1), (s2, e2) = range_map
    new_lower = max(rl, s1)
    new_upper = min(ru, e1)
    new_length = new_upper - new_lower
    new_map_lower = s2 + (new_lower - s1)
    
    result = ((new_lower, new_upper), (new_map_lower, new_map_lower + new_length))
    return result

clip_range_map_input((1, 1), ((0,5), (5,10)))

((1, 1), (6, 6))

In [252]:

def get_overlapping_range_maps(range_map_in, range_maps_out):
    return [clip_range_map_input(range_map_in[1], rm) for rm in range_maps_out if overlaps(range_map_in[1], rm[0])]

get_overlapping_range_maps(((10,20),(0, 10)), [((5,7), (15,17)), ((9,12), (14,17)), ((19, 20), (20, 21))])

[((5, 7), (15, 17)), ((9, 10), (14, 15))]

In [253]:
def compose_range_map(range_map1, range_map2):
    # precondition: (e,f) subset of (c,d)
    ((a,b), (c,d)) = range_map1
    ((e,f), (g,h)) = range_map2
    start_delta = e-c
    length = f-e
    start = a+start_delta
    return ((start, start+length), (g, h))

compose_range_map(((1, 3), (2, 4)), ((2,4), (8,10)))

((1, 3), (8, 10))

In [254]:
def fill_range_map_input_gaps(range, range_maps):
    l,u = range
    range_maps = sorted(range_maps, key=lambda rm: rm[0][0])
    lower_map_bound = range_maps[0][0][0]
    upper_map_bound = range_maps[-1][0][1]
    
    gaps = [((ub1+1, lb2-1),)*2 for ((_, ub1), _),((lb2, _), _) in pairwise(range_maps) if lb2-ub1>1]

    if lower_map_bound > l:
        gaps.append( ((l, lower_map_bound-1),)*2 )
    if upper_map_bound < u:
        gaps.append( ((upper_map_bound+1, u),)*2 )
    
    return sorted(range_maps + gaps, key=lambda rm: rm[0][0])

fill_range_map_input_gaps((0, 10), [((3,5), (18, 20)), ((0,2), (30,32))])

[((0, 2), (30, 32)), ((3, 5), (18, 20)), ((6, 10), (6, 10))]

In [255]:
def compose_maps(range_map, range_maps):
    overlapping_maps = get_overlapping_range_maps(range_map, range_maps)

    if not overlapping_maps: 
        return [range_map]
    
    overlapping_maps_no_gaps = fill_range_map_input_gaps(range_map[1], overlapping_maps)

    composition = [compose_range_map(range_map, rm) for rm in overlapping_maps_no_gaps]

    return composition 

compose_maps(get_seed_ranges(test_input)[0], get_sequential_ranges(test_input)[0])
 

[((79, 92), (81, 94))]

In [256]:
def follow_maps(input_maps, output_maps):
    input_maps = sorted(input_maps, key=lambda rm: rm[0][0])
    output_maps = sorted(output_maps, key=lambda rm: rm[0][0])
    return [new_map for input_map in input_maps for new_map in compose_maps(input_map, output_maps)]

follow_maps(get_seed_ranges(test_input), get_sequential_ranges(test_input)[0])

[((55, 67), (57, 69)), ((79, 92), (81, 94))]

In [261]:
def compose_sections(sections):
    range_maps = get_seed_ranges(sections)
    for section_maps in get_sequential_ranges(sections):
        range_maps = follow_maps(range_maps, section_maps)
    return range_maps

compose_sections(test_input)

[((55, 58), (86, 89)),
 ((59, 61), (94, 96)),
 ((62, 65), (56, 59)),
 ((66, 67), (97, 98)),
 ((79, 81), (82, 84)),
 ((82, 91), (46, 55)),
 ((92, 92), (60, 60))]

In [258]:
def get_min_output_val(range_maps):
    return min(lb for _,(lb,_) in range_maps)

get_min_output_val(compose_sections(test_input))

46

In [259]:
def get_score_2(sections):
    return get_min_output_val(compose_sections(sections))

get_score_2(test_input)

46

In [260]:
get_score_2(real_input)

79874951